In [ ]:

import pandas as pd
import plotly.express as px
import os, sys
from prophet import Prophet
from datetime import timedelta

# Pour import depuis src/
sys.path.append(os.path.abspath("../src"))
from weather_connector import get_weather_forecast


In [ ]:

# 📍 Ville cible
ville = "Tunis"
data = get_weather_forecast(ville)
df = pd.DataFrame(data)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df[["datetime", "temp"]].rename(columns={"datetime": "ds", "temp": "y"})
df.head()


In [ ]:

# 📈 Création et entraînement du modèle Prophet
model = Prophet()
model.fit(df)


In [ ]:

# 🔮 Prédire les prochaines 72 heures (~24 points de 3h)
future = model.make_future_dataframe(periods=24, freq='3H')
forecast = model.predict(future)
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail()


In [ ]:

# 📊 Visualisation des prédictions avec Plotly
fig = px.line()
fig.add_scatter(x=forecast["ds"], y=forecast["yhat"], name="Prévision")
fig.add_scatter(x=df["ds"], y=df["y"], name="Temp réelle")
fig.update_layout(title=f"🔮 Prévision de température – {ville}", xaxis_title="Date", yaxis_title="Température (°C)")
fig.show()


In [ ]:

# Comparaison sur les points connus
merged = pd.merge(df, forecast[["ds", "yhat"]], on="ds", how="left")
merged.dropna(inplace=True)
mae = abs(merged["y"] - merged["yhat"]).mean()
print(f"📉 MAE (Erreur absolue moyenne) : {mae:.2f} °C")
